In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [2]:
df = pd.read_csv('sonardataset.csv',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_52,x_53,x_54,x_55,x_56,x_57,x_58,x_59,x_60,Y
1,0.02,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.018,0.0084,0.009,0.0032,R
2,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.014,0.0049,0.0052,0.0044,R
3,0.0262,0.0582,0.1099,0.1083,0.0974,0.228,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.018,0.0244,0.0316,0.0164,0.0095,0.0078,R
4,0.01,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.015,0.0085,0.0073,0.005,0.0044,0.004,0.0117,R


In [3]:
df = df[df.iloc[:, 0] != 'x_1']
df = df.reset_index(drop=True)

In [4]:
###Split Features & Target
x = df.iloc[:, :-1]   
y = df.iloc[:, -1]    

In [5]:
x1 = x.apply(pd.to_numeric)

In [6]:
###encode target variables
le = LabelEncoder()
y = le.fit_transform(y) 

In [7]:
###feature normalization
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


In [8]:
###train test split
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled,
    y,
    test_size=0.2,
    random_state=42
)


In [9]:
###build ann model
model = Sequential()

model.add(Dense(16, activation='relu', input_shape=(60,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


C:\Users\AKSHAT\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
###compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [11]:
###train model
history = model.fit(
    x_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.5076 - loss: 0.7947 - val_accuracy: 0.3824 - val_loss: 0.7832
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5303 - loss: 0.7178 - val_accuracy: 0.4412 - val_loss: 0.7333
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5909 - loss: 0.6681 - val_accuracy: 0.4706 - val_loss: 0.6995
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6591 - loss: 0.6256 - val_accuracy: 0.5588 - val_loss: 0.6816
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6894 - loss: 0.5929 - val_accuracy: 0.5294 - val_loss: 0.6627
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7273 - loss: 0.5649 - val_accuracy: 0.5882 - val_loss: 0.6468
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7879 - loss: 0.5403 - val_accuracy: 0.6176 - val_loss: 0.6323
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8030 - loss: 0.5172 - val_accuracy: 0.6176 - val_loss: 0.6140


In [12]:
###prediction
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


In [13]:
###evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.7857142857142857
              precision    recall  f1-score   support

           0       0.90      0.73      0.81        26
           1       0.67      0.88      0.76        16

    accuracy                           0.79        42
   macro avg       0.79      0.80      0.78        42
weighted avg       0.81      0.79      0.79        42



In [14]:
!pip install keras-tuner


In [21]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [23]:
def build_model(hp):
    model = Sequential()

    for i in range(hp.Int('num_layers', min_value=1, max_value=3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=16),
            activation=hp.Choice('activation', ['relu', 'tanh'])
        ))

    
    model.add(Dense(1, activation='sigmoid'))

  
    learning_rate = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [24]:
### initialize
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='kt_dir',
    project_name='ann_tuning'
)


In [25]:
###Run Hyperparameter Search
tuner.search(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=20,
    batch_size=32,
    verbose=1
)


Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.8333333134651184

Best val_accuracy So Far: 0.9047619104385376
Total elapsed time: 00h 01m 30s


In [26]:
###Get Best Hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Optimal number of hidden layers:", best_hyperparameters.get('num_layers'))
print("Optimal activation function:", best_hyperparameters.get('activation'))
print("Optimal learning rate:", best_hyperparameters.get('learning_rate'))

for i in range(best_hyperparameters.get('num_layers')):
    print(f"Optimal neurons in layer {i+1}:", best_hyperparameters.get(f'units_{i}'))


Optimal number of hidden layers: 3
Optimal activation function: tanh
Optimal learning rate: 0.001
Optimal neurons in layer 1: 16
Optimal neurons in layer 2: 16
Optimal neurons in layer 3: 112


In [27]:
###Train Final Optimized Model
best_model = tuner.get_best_models(num_models=1)[0]

history = best_model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=20,
    batch_size=32,
    verbose=1
)


Epoch 1/20


C:\Users\AKSHAT\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.9036 - loss: 0.2976 - val_accuracy: 0.7857 - val_loss: 0.3670
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9157 - loss: 0.2706 - val_accuracy: 0.8333 - val_loss: 0.3573
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.8916 - loss: 0.2560 - val_accuracy: 0.8333 - val_loss: 0.3556
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8976 - loss: 0.2418 - val_accuracy: 0.8571 - val_loss: 0.3525
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8976 - loss: 0.2316 - val_accuracy: 0.8571 - val_loss: 0.3551
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9157 - loss: 0.2160 - val_accuracy: 0.8571 - val_loss: 0.3506
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9217 - loss: 0.2070 - val_accuracy: 0.8333 - val_loss: 0.3508
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9398 - loss: 0.1942 - val_accuracy: 0.8333 - val_loss: 0.3539
Epoch 9/20


In [28]:
###Evaluate Optimized Model
y_pred_opt = (best_model.predict(x_test) > 0.5).astype(int)

from sklearn.metrics import classification_report

print("Classification Report for Optimized ANN:")
print(classification_report(y_test, y_pred_opt))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Classification Report for Optimized ANN:
              precision    recall  f1-score   support

           0       0.88      0.85      0.86        26
           1       0.76      0.81      0.79        16

    accuracy                           0.83        42
   macro avg       0.82      0.83      0.83        42
weighted avg       0.84      0.83      0.83        42

